In [34]:
from deeplate.MMdata import MMData 
import os, stat
import subprocess
import numpy as np

In [39]:
#Do not add the last slash

#folder = '/Volumes/MusiqueHD/201710126_lowerbound'
folder = '/scicore/home/nimwegen/urchuegu/projects/prj_noise_across_conditions_main/cell_size/data/20170126/201710126_lowerbound'
#folder_analyzed = '/Users/guillaume/Desktop/PostdocBasel/PlateData/Analysis_'+os.path.basename(folder)+'/'
folder_save = '/scicore/home/nimwegen/witzg/DeepPlateAnalysis'
#weights_folder  = '/Users/guillaume/Desktop/PostdocBasel/DeepLearningData/Learn20170126/'
weights_folder  = '/scicore/home/nimwegen/witzg/DeepLearningData/Learn20170126_subset_stack2'

In [40]:
MMobj = MMData(folder = folder)
positions, well = MMobj.get_position_names()

In [41]:
#positions = positions[0:10]

In [42]:
len(positions)/4

57.0

In [43]:
MMobj = MMData(folder = folder)
print(MMobj.channels)

['GFP', 'DIA (GFP)']


In [44]:
#based on the above outpout set fluo and bright field channels
fluo_ch = 0
bf_ch = 1

In [45]:
folder_analyzed = folder_save+'/Analysis_'+os.path.basename(folder)+'/'
folder_logs = folder_save+'/Analysis_'+os.path.basename(folder)+'/logs/'
if not os.path.exists(folder_analyzed):
    os.makedirs(folder_logs)
if not os.path.exists(folder_logs):
    os.makedirs(folder_logs)

In [46]:
basename = os.path.basename(folder)
num_in_group = 5
pos_seq = np.arange(len(positions))
partitioned = [pos_seq[i:i+num_in_group] for i  in range(0, len(pos_seq), num_in_group)]
for p in partitioned:
    filename = folder_analyzed+basename+'_'+str(p[0])+'_'+str(p[-1])+'.sh'

    f = open(filename, "w")
    
    #regular CPU
    '''f.write('#!/bin/bash\n')
    f.write('#SBATCH --job-name=test_JOB2 \n')
    f.write('#SBATCH --time=01:00:00 \n')
    f.write('#SBATCH --mem=8G \n')
    f.write('#SBATCH --qos=6hours \n')
    f.write('#SBATCH -e '+folder_logs+'error_'+basename+'_'+str(p[0])+'_'+str(p[-1])+'.log \n')
    f.write('#SBATCH -o '+folder_logs+'output_'+basename+'_'+str(p[0])+'_'+str(p[-1])+'.log \n')
    f.write('module load Python/3.5.2-goolf-1.7.20 \n')
    f.write('source /scicore/home/nimwegen/witzg/DeepPlateSegmenter/venv-deeplate/bin/activate \n')
    f.write('python deep_plate_analysis.py "'+
            folder+'" "'+weights_folder+'" "'+folder_analyzed+'" '+str(p[0])+' '+str(num_in_group)+' '+str(bf_ch)+' '+str(fluo_ch)+' \n')
    f.close()'''
    
    #GPU
    f.write('#!/bin/bash\n')
    f.write('#SBATCH --job-name=test_JOB-gpu \n')
    f.write('#SBATCH --time=01:00:00 \n')
    f.write('#SBATCH --mem-per-cpu=8G \n')
    f.write('#SBATCH --ntasks=1 \n')
    f.write('#SBATCH --cpus-per-task=1 \n')
    f.write('#SBATCH --qos=6hours \n')
    f.write('#SBATCH --partition=k80 \n')
    f.write('#SBATCH --gres=gpu:1 \n')
    f.write('#SBATCH -e '+folder_logs+'error_'+basename+'_'+str(p[0])+'_'+str(p[-1])+'.log \n')
    f.write('#SBATCH -o '+folder_logs+'output_'+basename+'_'+str(p[0])+'_'+str(p[-1])+'.log \n')
    f.write('module load Python/3.5.2-goolf-1.7.20 \n')
    f.write('module load cuDNN \n')
    f.write('source /scicore/home/nimwegen/witzg/DeepPlateSegmenter/venv-deeplate-gpu/bin/activate \n')
    f.write('python deep_plate_analysis_multiD.py "'+
            folder+'" "'+weights_folder+'" "'+folder_analyzed+'" '+str(p[0])+' '+str(num_in_group)+' '+str(bf_ch)+' '+str(fluo_ch)+' \n')
    f.close()
    
    
    os.chmod(filename, stat.S_IRWXU)
    
    subprocess.call(['sbatch',filename])